[View in Colaboratory](https://colab.research.google.com/github/dingaaling/roadExtract/blob/master/cnn.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [1]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [3]:
import os, cv2, sklearn, random
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras import backend as K
from keras.regularizers import l2
from keras.layers import LeakyReLU
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
root_dir = os.getcwd() + "/drive/road_extract/data/sample/"
image_dir = root_dir + "train_sample/"
mask_dir = root_dir + "mask_sample/"

imgs_list, mask_imgs_list = [], []


for i, im in enumerate(os.listdir(image_dir)):
  if i < 200:
    imPath = image_dir + im
    maskPath = mask_dir + im.split('_')[0] + "_mask.png"

    imgs_list.append(load_image(imPath))
    mask_imgs_list.append(load_gray_image(maskPath))


imgs = np.asarray(imgs_list)
mask_imgs = np.asarray(mask_imgs_list)

In [0]:
del imgs_list
del mask_imgs_list

In [6]:
print(imgs.shape)
print(mask_imgs.shape)

(200, 1024, 1024, 3)
(200, 1024, 1024)


# Create Array of Windows and Labels

In [0]:
patch_w, patch_h, patch_stride, patch_pad = 32, 32, 32, 0
img_patches = img_crop(imgs[0],patch_w,patch_h,patch_stride,patch_pad)
for i in range(1,imgs.shape[0]):
  img_patches = np.concatenate((img_patches,img_crop(imgs[i],patch_w,patch_h,patch_stride,patch_pad)))
X = img_patches
Y = np.asarray([load_mask(mask_imgs[i],patch_w,patch_h,patch_stride,0) for i in range(mask_imgs.shape[0])])
Y = np.reshape(Y,-1)

In [8]:
print(X.shape)
print(Y.shape)
print(sum(Y))

(192200, 32, 32, 3)
(192200,)
8835


In [0]:
del img_patches
del imgs
del mask_imgs

In [0]:
Y_2d = np_utils.to_categorical(Y, num_classes=2)

In [11]:
X_road = np.zeros((np.count_nonzero(Y_2d[:,1]),X.shape[1],X.shape[2],X.shape[3]))
Y_road_2d = np.zeros((np.count_nonzero(Y_2d[:,1]),Y_2d.shape[1]))
X_nonroad = np.zeros(((X.shape[0]-np.count_nonzero(Y_2d[:,1])),X.shape[1],X.shape[2],X.shape[3]))
Y_nonroad_2d = np.zeros(((X.shape[0]-np.count_nonzero(Y_2d[:,1])),Y_2d.shape[1]))
print(X_road.shape)
print(Y_road_2d.shape)
print(X_nonroad.shape)
print(Y_nonroad_2d.shape)
road_ind = 0
nonroad_ind = 0
for i in range(X.shape[0]):
  if Y[i]>0.5:
    X_road[road_ind,:,:,:] = X[i,:,:,:]
    Y_road_2d[road_ind,:] = Y_2d[i,:]
    road_ind = road_ind+1
  else:
    X_nonroad[nonroad_ind,:,:,:] = X[i,:,:,:]
    Y_nonroad_2d[nonroad_ind,:] = Y_2d[i,:]
    nonroad_ind = nonroad_ind+1

(8835, 32, 32, 3)
(8835, 2)
(183365, 32, 32, 3)
(183365, 2)


In [0]:
X_nonroad_sample = np.zeros(X_road.shape)
Y_2d_nonroad_sample = np.zeros(Y_road_2d.shape)
rand_ind_nonroad = random.sample(np.ndarray.tolist(np.arange(X_nonroad.shape[0])),np.count_nonzero(Y_2d[:,1]))
x_ind = 0
y_ind = 0
for ind in rand_ind_nonroad:
  X_nonroad_sample[x_ind,:,:,:] = X_nonroad[ind,:,:,:]
  Y_2d_nonroad_sample[y_ind,:] = Y_nonroad_2d[ind,:]
  x_ind = x_ind+1
  y_ind = y_ind+1

In [0]:
X_even = np.zeros((2*np.count_nonzero(Y_2d[:,1]),X.shape[1],X.shape[2],X.shape[3]))
Y_even_2d = np.zeros((2*np.count_nonzero(Y_2d[:,1]),Y_2d.shape[1]))
X_even[0:np.count_nonzero(Y_2d[:,1]),:,:,:] = X_road
Y_even_2d[0:np.count_nonzero(Y_2d[:,1]),:] = Y_road_2d
X_even[np.count_nonzero(Y_2d[:,1]):2*np.count_nonzero(Y_2d[:,1]),:,:,:] = X_nonroad_sample
Y_even_2d[np.count_nonzero(Y_2d[:,1]):2*np.count_nonzero(Y_2d[:,1]),:] = Y_2d_nonroad_sample

In [0]:
del X, Y, X_road, Y_road_2d, X_nonroad, Y_nonroad_2d

In [15]:
print(X_even.shape)
print(Y_even_2d.shape)

(17670, 32, 32, 3)
(17670, 2)


# CNN Model

In [16]:
pool_size = (2, 2)
reg = 1e-6 # L2 regularization factor (used on weights, but not biases)
nb_classes = 2

model = Sequential()

model.add(Convolution2D(64, 5, 5, # 64 5x5 filters
                        border_mode='same',
                        input_shape=(patch_w,patch_h, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Convolution2D(128, 3, 3, # 128 3x3 filters
                        border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Convolution2D(256, 3, 3, # 256 3x3 filters
                        border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, W_regularizer=l2(reg))) # Fully connected layer (128 neurons)
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))

model.add(Dense(nb_classes, W_regularizer=l2(reg)))
model.add(Activation('softmax'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(32, 32, 3..., padding="same")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same")`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="same")`
/us

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        4864      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 128)         0         
__________

In [0]:
adam = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])

In [19]:
model.fit(X_even,Y_even_2d, batch_size=128, epochs=200)

Epoch 1/200
17670/17670 [==============================] - 10s 560us/step - loss: 6.8670 - acc: 0.5025
Epoch 2/200
17670/17670 [==============================] - 6s 352us/step - loss: 6.1798 - acc: 0.5198
Epoch 3/200
17670/17670 [==============================] - 6s 354us/step - loss: 5.0631 - acc: 0.5244
Epoch 4/200
17670/17670 [==============================] - 6s 354us/step - loss: 3.0822 - acc: 0.5423
Epoch 5/200
12160/17670 [===================>..........] - ETA: 1s - loss: 2.1806 - acc: 0.5521

17670/17670 [==============================] - 6s 350us/step - loss: 2.1215 - acc: 0.5508
Epoch 6/200
17670/17670 [==============================] - 6s 352us/step - loss: 1.7671 - acc: 0.5476
Epoch 7/200
17670/17670 [==============================] - 6s 349us/step - loss: 1.5000 - acc: 0.5611
Epoch 8/200
17670/17670 [==============================] - 6s 348us/step - loss: 1.2980 - acc: 0.5632
Epoch 9/200
16896/17670 [===========================>..] - ETA: 0s - loss: 1.1491 - acc: 0.5641

17670/17670 [==============================] - 6s 349us/step - loss: 1.1463 - acc: 0.5641
Epoch 10/200
17670/17670 [==============================] - 6s 348us/step - loss: 1.0593 - acc: 0.5699
Epoch 11/200
17670/17670 [==============================] - 6s 349us/step - loss: 0.9931 - acc: 0.5759
Epoch 12/200
17670/17670 [==============================] - 6s 345us/step - loss: 0.9253 - acc: 0.5835
Epoch 13/200
17152/17670 [============================>.] - ETA: 0s - loss: 0.8759 - acc: 0.5823

17670/17670 [==============================] - 6s 351us/step - loss: 0.8760 - acc: 0.5814
Epoch 14/200
17670/17670 [==============================] - 6s 348us/step - loss: 0.8504 - acc: 0.5879
Epoch 15/200
17670/17670 [==============================] - 6s 349us/step - loss: 0.8184 - acc: 0.5892
Epoch 16/200
17670/17670 [==============================] - 6s 352us/step - loss: 0.7954 - acc: 0.5954
Epoch 17/200
17664/17670 [============================>.] - ETA: 0s - loss: 0.7867 - acc: 0.5957

17670/17670 [==============================] - 6s 349us/step - loss: 0.7865 - acc: 0.5959
Epoch 18/200
17670/17670 [==============================] - 6s 354us/step - loss: 0.7553 - acc: 0.6062
Epoch 19/200
17670/17670 [==============================] - 6s 353us/step - loss: 0.7439 - acc: 0.6115
Epoch 20/200
17670/17670 [==============================] - 6s 351us/step - loss: 0.7307 - acc: 0.6140
Epoch 21/200
17670/17670 [==============================] - 6s 348us/step - loss: 0.7249 - acc: 0.6169


...

17670/17670 [==============================] - 6s 352us/step - loss: 0.4308 - acc: 0.7988
Epoch 184/200
17670/17670 [==============================] - 6s 354us/step - loss: 0.4306 - acc: 0.8010
Epoch 185/200
17670/17670 [==============================] - 6s 352us/step - loss: 0.4320 - acc: 0.8031
Epoch 186/200
17670/17670 [==============================] - 6s 351us/step - loss: 0.4278 - acc: 0.8014
Epoch 187/200
16768/17670 [===========================>..] - ETA: 0s - loss: 0.4264 - acc: 0.8009

17670/17670 [==============================] - 6s 352us/step - loss: 0.4252 - acc: 0.8007
Epoch 188/200
17670/17670 [==============================] - 6s 351us/step - loss: 0.4248 - acc: 0.8031
Epoch 189/200
17670/17670 [==============================] - 6s 351us/step - loss: 0.4263 - acc: 0.8024
Epoch 190/200
17670/17670 [==============================] - 6s 347us/step - loss: 0.4261 - acc: 0.8034
Epoch 191/200
17024/17670 [===========================>..] - ETA: 0s - loss: 0.4220 - acc: 0.8052

17670/17670 [==============================] - 6s 353us/step - loss: 0.4219 - acc: 0.8054
Epoch 192/200
17670/17670 [==============================] - 6s 351us/step - loss: 0.4221 - acc: 0.8065
Epoch 193/200
17670/17670 [==============================] - 6s 350us/step - loss: 0.4223 - acc: 0.8049
Epoch 194/200
17670/17670 [==============================] - 6s 351us/step - loss: 0.4234 - acc: 0.8058
Epoch 195/200
17670/17670 [==============================] - 6s 351us/step - loss: 0.4193 - acc: 0.8062


Epoch 196/200
17670/17670 [==============================] - 6s 354us/step - loss: 0.4245 - acc: 0.8053
Epoch 197/200
17670/17670 [==============================] - 6s 350us/step - loss: 0.4211 - acc: 0.8072
Epoch 198/200
17670/17670 [==============================] - 6s 353us/step - loss: 0.4203 - acc: 0.8059
Epoch 199/200
17670/17670 [==============================] - 6s 352us/step - loss: 0.4213 - acc: 0.8050
Epoch 200/200
10880/17670 [=================>............] - ETA: 2s - loss: 0.4169 - acc: 0.8106

17670/17670 [==============================] - 6s 350us/step - loss: 0.4199 - acc: 0.8071


# Helper Functions

In [0]:
def img_crop(im, w, h, stride, padding):
    """ Crop an image into patches, taking into account mirror boundary conditions. """
    assert len(im.shape) == 3, 'Expected RGB image.'
    num_patches = int((im.shape[0]-w)*(im.shape[1]-h)/(stride*stride))
    np_patches = np.ndarray((num_patches,w,h,3))
    imgwidth = im.shape[0]
    half_w = int(w/2)
    half_h = int(h/2)
    imgheight = im.shape[1]
#     im = np.lib.pad(im, ((padding, padding), (padding, padding), (0,0)), 'reflect')
    ind = 0
    for i in range(int(padding+(h/2)),int(imgheight+padding-(h/2)),stride):
        for j in range(int(padding+(w/2)),int(imgwidth+padding-(w/2)),stride):
            im_patch = im[j-half_w-padding:j+half_w+padding, i-half_h-padding:i+half_h+padding, :]
            np_patches[ind,:,:,:] = im_patch
            ind = ind+1
    return np_patches
      
def create_patches(X, patch_size, stride, padding):
    img_patches = np.asarray([img_crop(X[i], patch_size, patch_size, stride, padding) for i in range(X.shape[0])])
    # Linearize list
    img_patches = img_patches.reshape(-1, img_patches.shape[2], img_patches.shape[3], img_patches.shape[4])
    return img_patches
  
def load_mask(img,w,h,stride,padding):
    list_patches = []
    imgwidth = img.shape[0]
    imgheight = img.shape[1]
    for i in range(int(padding+(h/2)),int(imgheight+padding-(h/2)),stride):
      for j in range(int(padding+(w/2)),int(imgwidth+padding-(w/2)),stride):
        if img[j,i]>0.5:
          list_patches.append(1)
        else:
          list_patches.append(0)
    return list_patches

In [0]:
def load_image(fileName):
    return mpimg.imread(fileName)
  
def load_gray_image(infilename):
    img = cv2.imread(infilename)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    resized_image = cv2.resize(gray, (72,72)) 
    return gray
  
def grey_scale_conversion(img):
    new_img = np.zeros((img.shape[0],img.shape[1]))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i,j,1] == 1.0:
                new_img[i,j] = 1
    return new_img
  
def grey_scale_conversion(img):
    new_img = np.zeros((img.shape[0],img.shape[1]))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i,j,1] == 1.0:
                new_img[i,j] = 1
    return new_img
  

def pad_image(data, padding):
    """
    Extend the canvas of an image. Mirror boundary conditions are applied.
    """
    if len(data.shape) < 3:
        # Greyscale image (ground truth)
        data = np.lib.pad(data, ((padding, padding), (padding, padding)), 'reflect')
    else:
        # RGB image
        data = np.lib.pad(data, ((padding, padding), (padding, padding), (0,0)), 'reflect')
    return data
    
def img_crop_gt(im, w, h, stride):
    """ Crop an image into patches (this method is intended for ground truth images). """
    assert len(im.shape) == 2, 'Expected greyscale image.'
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    for i in range(0,imgheight,stride):
        for j in range(0,imgwidth,stride):
            im_patch = im[j:j+w, i:i+h]
            list_patches.append(im_patch)
    return list_patches
    
      
    
def create_patches_gt(X, patch_size, stride):
    img_patches = np.asarray([img_crop_gt(X[i], patch_size, patch_size, stride) for i in range(X.shape[0])])
    # Linearize list
    img_patches = img_patches.reshape(-1, img_patches.shape[2], img_patches.shape[3])
    return img_patches
    
def group_patches(patches, num_images):
    return patches.reshape(num_images, -1)

def extract_img_features(filename, stride):
    img = load_image(filename)
    img_patches = img_crop(img, patch_size, patch_size, stride, padding)
    X = np.asarray([img_patches[i] for i in range(len(img_patches))])
    return X